
# Source localization with single dipole fit


The aim of this lecture is to show you how to do sequential and
fixed dipole fitting with MNE-Python.

In [1]:
%matplotlib qt
import matplotlib.pyplot as plt

import os
import numpy as np
import mne

mne.set_log_level('WARNING')

# Change the following path to where the folder ds000117-practical is on your disk
data_path = os.path.expanduser("~\\Downloads\\meeg\\ds000117-practical\\")  # this works and is user-independent

raw_fname = os.path.join(data_path,
    'derivatives\\meg_derivatives\\sub-01\\ses-meg\\meg\\sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg.fif')

epochs_fname = raw_fname.replace('_meg.fif', '-epo.fif')

In [10]:
subjects_dir = os.path.join(data_path, 'derivatives\\freesurfer-reconall\\')
subject = 'sub-01'
trans_fname = os.path.join(data_path,
    'derivatives\\meg_derivatives\\sub-01\\ses-meg\\meg\\sub-01-trans.fif')
bem_fname = os.path.join(data_path,
    'derivatives\\meg_derivatives\\sub-01\\ses-meg\\meg\\sub-01-bem.fif')
#fname_surf_lh = os.path.join(subjects_dir, subject, 'surf', 'lh.white')

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>Check that the head geometry and sensor alignment is correct</li>
    </ul>
</div>

You will need to use the `mne.viz.plot_alignment` function that can take as input a `bem` parameter of type `ConductorModel`

In [11]:
bem = mne.bem.read_bem_solution(bem_fname)
bem

<ConductorModel  |  BEM (1 layer)>

In [12]:
# TODO
info = mne.io.read_info(epochs_fname)
fig = mne.viz.plot_alignment(info, trans_fname, subject=subject, dig=True,
                             subjects_dir=subjects_dir, bem=bem, verbose=True);

Could not find the surface for head in the provided BEM model, looking in the subject directory.
Using outer_skin.surf for head surface.
Getting helmet for system 306m


## Let's localize the N170m (using MEG only) using dipole fitting

In [13]:
epochs = mne.read_epochs(epochs_fname)

In [14]:
epochs.pick_types(meg=True, eeg=False)

<EpochsFIF  |   144 events (all good), -0.5 - 2 sec, baseline [-0.2, 0], ~259.9 MB, data loaded,
 'face/famous/first': 24
 'face/famous/immediate': 10
 'face/famous/long': 14
 'face/unfamiliar/first': 25
 'face/unfamiliar/immediate': 11
 'face/unfamiliar/long': 10
 'scrambled/first': 25
 'scrambled/immediate': 14
 'scrambled/long': 11>

In [43]:
cov = mne.compute_covariance(epochs, rank='info')  ## tmax = "the end time for baseline", which is 0 (defined in <epochs>)

In [45]:
cov

<Covariance  |  size : 306 x 306, n_samples : 108144, data : [[ 1.15947527e-23  4.29511821e-24  2.28686851e-25 ...  3.43366826e-24
   4.61773660e-25  1.14859436e-25]
 [ 4.29511821e-24  1.76799058e-23  1.48342486e-25 ...  5.00592768e-24
  -4.47451934e-25  1.77299127e-25]
 [ 2.28686851e-25  1.48342486e-25  3.00107667e-26 ...  3.40605402e-25
   9.40284008e-26  6.93747958e-27]
 ...
 [ 3.43366826e-24  5.00592768e-24  3.40605402e-25 ...  3.84771054e-23
   4.86696597e-24  8.02671470e-26]
 [ 4.61773660e-25 -4.47451934e-25  9.40284008e-26 ...  4.86696597e-24
   1.64282201e-23  4.27698983e-26]
 [ 1.14859436e-25  1.77299127e-25  6.93747958e-27 ...  8.02671470e-26
   4.27698983e-26  2.64156381e-26]]>

In [16]:
evoked_face = epochs['face'].average()
evoked_scrambled = epochs['scrambled'].average()

In [46]:
contrast = mne.combine_evoked([evoked_face, evoked_scrambled], [0.5, -0.5])
contrast.crop(None, 0.2)
contrast.plot();

In [38]:
# Fit a dipole using a sequential (time-varying position and orientation) fit
contrast_crop = contrast.copy().crop(0.05, 0.06)
dip, residual = mne.fit_dipole(contrast_crop, cov, bem_fname,
                               trans_fname)
print(dip)

<Dipole  |  n_times : 4, tmin : 0.050, tmax : 0.060>


In [39]:
# Look at our result
print(dip.gof)

[25.13976224 25.506638   25.23810197 22.42646327]


In [40]:
dip.plot_locations(subject=subject, trans=trans_fname,
                   subjects_dir=subjects_dir, mode='orthoview');

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>Try to improve the dipole fit by using a subselection of channels [0] </li>
      <li>What is the improvement in max GOF from using a subselection of channels?</li>     
      <li>Should you also try to maximize GOF?</li>
    </ul>
</div>

Tips and tricks:

   - [0] Subselect channels (think about ``mne.read_selection(...)``, ``evoked.pick_channels(...)``)

Trying the same with EEG enabled doesn't seem to do the trick, because <epochs> only has MEG sensors.

In [31]:
# TODO
selection = mne.read_selection('Left', info=contrast.info)
# Fit a dipole using a sequential (time-varying position and orientation) fit
dip, residual = \
    mne.fit_dipole(contrast_crop.copy().pick_channels(selection),
                   cov, bem_fname, trans_fname)
print(dip)

<ipython-input-31-df7a1766e896>:6: RuntimeWarning: Projection vector "EOG-planar--0.200-0.200-PCA-01" has magnitude 0.71 (should be unity), applying projector with 102/204 of the original channels available may be dangerous, consider recomputing and adding projection vectors for channels that are eventually used. If this is intentional, consider using info.normalize_proj()
  cov, bem_fname, trans_fname)
<ipython-input-31-df7a1766e896>:6: RuntimeWarning: Projection vector "EOG-planar--0.200-0.200-PCA-02" has magnitude 0.84 (should be unity), applying projector with 102/204 of the original channels available may be dangerous, consider recomputing and adding projection vectors for channels that are eventually used. If this is intentional, consider using info.normalize_proj()
  cov, bem_fname, trans_fname)
<ipython-input-31-df7a1766e896>:6: RuntimeWarning: Projection vector "EOG-planar--0.200-0.200-PCA-03" has magnitude 0.72 (should be unity), applying projector with 102/204 of the origina

<Dipole  |  n_times : 6, tmin : 0.173, tmax : 0.190>


In [32]:
print(dip.gof)

[33.77001166 40.24123163 40.96372156 30.0573477  34.03894796 54.24222316]


In [33]:
dip.plot_locations(subject=subject, trans=trans_fname,
                   subjects_dir=subjects_dir, mode='orthoview');